In [9]:
import numpy as np
import torch
import torch.nn as nn
import torchbnn as bnn  # torchbnn library for BNN layers
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [10]:
class BayesianNN(nn.Module):
    def __init__(self, input_dim):
        super(BayesianNN, self).__init__()
        # Define prior parameters
        prior_mu = 0.0  # Mean of the prior distribution
        prior_sigma = 0.1  # Standard deviation of the prior distribution
        
        # Initialize Bayesian layers with the specified priors
        self.fc1 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=input_dim, out_features=64)
        self.fc2 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=64, out_features=32)
        self.fc3 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=32, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [11]:
# Training function
def train_bayesian_nn(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred.squeeze(), y_batch)  # Squeeze to match dimensions
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [12]:
# Evaluation function
def evaluate_bayesian_nn(model, loader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for X_batch, _ in loader:
            y_pred = model(X_batch)
            all_preds.extend(y_pred.round().squeeze().cpu().numpy())
    return np.array(all_preds)

In [13]:
# Function to run BNN for each value of n
def run_bayesian_nn(n):
    # Load data
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Split data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Convert data to PyTorch tensors and create DataLoaders
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    
    # Initialize the model, loss function, and optimizer
    input_dim = X_train.shape[1]
    model = BayesianNN(input_dim)
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    for epoch in range(30):  # Adjust epochs as needed
        train_loss = train_bayesian_nn(model, train_loader, criterion, optimizer)
        print(f"Epoch {epoch+1} - Training Loss: {train_loss:.4f}")
    
    # Validate the model
    y_val_pred = evaluate_bayesian_nn(model, val_loader)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Validation Accuracy for n={n}: {val_accuracy:.4f}")
    
    # Test the model
    y_test_pred = evaluate_bayesian_nn(model, test_loader)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy for n={n}: {test_accuracy:.4f}")
    
    return test_accuracy

# Run for each n value and collect accuracies
results = []
possible_n_vals = [9]
for n in tqdm(possible_n_vals):
    accuracy = run_bayesian_nn(n)
    results.append((n, accuracy))

print("Accuracies across different n values:", results)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 - Training Loss: 0.6980
Epoch 2 - Training Loss: 0.6971
Epoch 3 - Training Loss: 0.6961
Epoch 4 - Training Loss: 0.6945
Epoch 5 - Training Loss: 0.6936
Epoch 6 - Training Loss: 0.6944
Epoch 7 - Training Loss: 0.6947
Epoch 8 - Training Loss: 0.6931
Epoch 9 - Training Loss: 0.6946
Epoch 10 - Training Loss: 0.6945
Epoch 11 - Training Loss: 0.6933
Epoch 12 - Training Loss: 0.6935
Epoch 13 - Training Loss: 0.6935
Epoch 14 - Training Loss: 0.6935
Epoch 15 - Training Loss: 0.6935
Epoch 16 - Training Loss: 0.6924
Epoch 17 - Training Loss: 0.6925
Epoch 18 - Training Loss: 0.6908
Epoch 19 - Training Loss: 0.6887
Epoch 20 - Training Loss: 0.6867
Epoch 21 - Training Loss: 0.6826
Epoch 22 - Training Loss: 0.6737
Epoch 23 - Training Loss: 0.6593
Epoch 24 - Training Loss: 0.6412
Epoch 25 - Training Loss: 0.6263
Epoch 26 - Training Loss: 0.6007
Epoch 27 - Training Loss: 0.5721
Epoch 28 - Training Loss: 0.5482
Epoch 29 - Training Loss: 0.5165


100%|██████████| 1/1 [00:22<00:00, 22.49s/it]

Epoch 30 - Training Loss: 0.4857
Validation Accuracy for n=9: 0.7744
Test Accuracy for n=9: 0.7813
Accuracies across different n values: [(9, 0.7812962962962963)]
